In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

In [5]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

## 데이터 스케일링

In [6]:
scaler=MinMaxScaler()
df = origin1.copy()
df['유통업체minmax']=scaler.fit_transform(df[['행정동별유통업체수']])
df['공공체육시설minmax']=scaler.fit_transform(df[['공공체육시설_수']])
df['문화시설minmax']=scaler.fit_transform(df[['문화시설_수']])
df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,유통업체minmax,공공체육시설minmax,문화시설minmax,면적minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,4552.529183,0.0,0.256410,0.055556,0.270833
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,7611.382114,0.0,0.205128,0.120370,0.115741
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,1728.859060,0.0,0.051282,0.287037,0.145833
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,4130.837004,0.0,0.128205,0.046296,0.236111
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,1990.755355,0.0,0.128205,0.101852,1.000000


## 파생변수(`여가생활지수`) 생성

In [7]:
df['여가생활지수'] = (df['공공체육시설minmax'] + df['문화시설minmax'] + df['행정동별유통업체수']) / df['면적minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,유통업체minmax,공공체육시설minmax,문화시설minmax,면적minmax,여가생활지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,32,0,3,4552.529183,0.0,0.256410,0.055556,0.270833,1.151874
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,27,0,12,7611.382114,0.0,0.205128,0.120370,0.115741,2.812308
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,16,0,5,1728.859060,0.0,0.051282,0.287037,0.145833,2.319902
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,10,34,0,5,4130.837004,0.0,0.128205,0.046296,0.236111,0.739065
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,10,54,0,5,1990.755355,0.0,0.128205,0.101852,1.000000,0.230057


## 파생 변수의 값 정렬

In [8]:
df['여가생활지수'].value_counts().sort_index()

여가생활지수
0.000000      37
0.046784       1
0.072072       1
0.074592       1
0.078818       1
              ..
203.294118     1
249.142857     1
259.200000     1
435.076923     1
inf            1
Name: count, Length: 360, dtype: int64

## 결측치 확인

In [9]:
df['여가생활지수'].isna().sum()

1

In [10]:
# df['여가생활지수'].filln(0)
# df['여가생활지수'].isnu().sum()

In [10]:
df['여가생활지수'].unique()

array([1.15187377e+00, 2.81230769e+00, 2.31990232e+00, 7.39064857e-01,
       2.30056980e-01, 6.81656805e+00, 2.51282051e+00, 1.54838710e+00,
       3.00377358e+01, 7.07027027e+01, 1.73202797e+01, 1.54606742e+01,
       1.14538462e+02, 0.00000000e+00,            inf,            nan,
       7.20000000e+01, 3.72222222e+01, 1.28100883e+02, 6.89473684e+01,
       2.63736264e-01, 1.65554799e+00, 2.03294118e+02, 7.02702703e+01,
       4.35076923e+02, 8.57142857e-01, 7.03261538e+01, 2.74125874e+00,
       1.00000000e+00, 2.66666667e+00, 2.59200000e+02, 8.86153846e-01,
       4.78632479e-01, 7.94871795e-01, 8.08266361e-01, 1.23076923e+00,
       1.38461538e+00, 7.38461538e+00, 4.21573034e+01, 1.98888564e-01,
       2.56470588e+01, 1.26984127e-01, 5.42335362e-01, 3.39299611e+00,
       1.77959184e+01, 4.65290807e-01, 7.63925729e-01, 4.88888889e+01,
       9.09473684e+01, 6.51583710e-01, 3.47826087e-01, 7.35459662e-01,
       4.67836257e-02, 4.73916888e-01, 1.21212121e-01, 9.19148936e+00,
      

In [11]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도', '유통업체minmax', '공공체육시설minmax', '문화시설minmax',
       '면적minmax', '여가생활지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [12]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','공공체육시설_수','문화시설_수','면적 (k㎡)', '행정동별유통업체수', '공공체육시설minmax','문화시설minmax','면적minmax', '유통업체minmax', '여가생활지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적 (k㎡),행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.270833,0.0,1.151874
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.115741,0.0,2.812308
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.145833,0.0,2.319902
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.236111,0.0,0.739065
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,1.000000,0.0,0.230057


## 지수 파일 생성

In [13]:
df1.to_excel('여가생활지수1.xlsx', index=False)

In [14]:
df2=read_excel('여가생활지수1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적 (k㎡),행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.270833,0.0,1.151874
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.115741,0.0,2.812308
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.145833,0.0,2.319902
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.236111,0.0,0.739065
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,1.000000,0.0,0.230057


## 이상치 `inf`, `-inf` 처리

In [15]:
# DataFrame(df2)에서 '여가생활지수' 열을 선택
accessibility_series = df2['여가생활지수']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '여가생활지수' 열을 DataFrame에 할당
df2['여가생활지수'] = accessibility_series

In [16]:
df2.to_excel("여가생활지수1.xlsx", index=False)

In [17]:
df2 = read_excel("여가생활지수1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적 (k㎡),행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.270833,0.0,1.151874
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.115741,0.0,2.812308
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.145833,0.0,2.319902
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.236111,0.0,0.739065
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,1.000000,0.0,0.230057


## 지수 정규화

In [18]:
df2['여가생활지수minmax'] = scaler.fit_transform(df2[['여가생활지수']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,공공체육시설_수,문화시설_수,면적 (k㎡),행정동별유통업체수,공공체육시설minmax,문화시설minmax,면적minmax,유통업체minmax,여가생활지수,여가생활지수minmax
0,11110,종로구,1111051500,청운효자동,10,6,2.57,0,0.256410,0.055556,0.270833,0.0000,1.151874,0.002648
1,11110,종로구,1111053000,사직동,8,13,1.23,0,0.205128,0.120370,0.115741,0.0000,2.812308,0.006464
2,11110,종로구,1111054000,삼청동,2,31,1.49,0,0.051282,0.287037,0.145833,0.0000,2.319902,0.005332
3,11110,종로구,1111055000,부암동,5,5,2.27,0,0.128205,0.046296,0.236111,0.0000,0.739065,0.001699
4,11110,종로구,1111056000,평창동,5,11,8.87,0,0.128205,0.101852,1.000000,0.0000,0.230057,0.000529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,0,2,0.67,1,0.000000,0.018519,0.050926,0.0625,20.000000,0.045969
402,11740,강동구,1174066000,성내제3동,0,1,0.71,1,0.000000,0.009259,0.055556,0.0625,18.166667,0.041755
403,11740,강동구,1174068500,길동,1,1,1.61,2,0.025641,0.009259,0.159722,0.1250,12.740245,0.029283
404,11740,강동구,1174069000,둔촌제1동,0,0,0.92,0,0.000000,0.000000,0.079861,0.0000,0.000000,0.000000


## 최종 데이터 저장

In [ ]:
df2.to_excel('여가생활지수1.xlsx',index=False)